In [ ]:
import os
import pandas as pd
import numpy as np
import math
from itertools import combinations
from statsmodels.tsa.api import VAR
from scipy.stats import pearsonr
from statsmodels.tsa.statespace.sarimax import SARIMAX
def log_transf(x):
  if (x != 0 and x != np.nan):
    return np.log(x)
  if (x == 0 and x != np.nan):
    return 0
  if x == np.nan:
    return np.nan

RANDOM_STATE = 12345

def invert_transformation_period(df_train, df_forecast, period):
  n = len(df_forecast)
  columns = df_train.columns
  for col in columns:
    train = df_train[col].to_list()
    for i in range(n):
      new_data = df_forecast[col].iloc[i] + train[-period]
      train.append(new_data)
    df_fc = pd.DataFrame(train[-n:], columns=[col])
    df_fc.index = df_forecast.index
  return df_fc

# activate R magic
%load_ext rpy2.ipython

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# **SARIMAX**

In [ ]:
def sarimax_window(col_core, exo_features, n_train, n_test, p, window):
  df_US = pd.read_csv('COVID19_DATA.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  df_US = df_US.drop(columns=['date'])
  df_US_log = pd.DataFrame()
  for col in df_US.columns:
    df_US_log[col] = df_US[col].apply(lambda x: log_transf(x))
  df_US_log
  total_features = ['icu_patients', 'hosp_patients', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated']
  df_sarimax14 = df_US_log[[col_core]][14:]
  df_exog14 = df_US_log[total_features][0:-14]
  df_exog14.index = df_sarimax14.index
  df_sarimax_old14 = df_sarimax14.loc[df_US_log[38:].index]
  df_current_old14 = df_sarimax_old14[int(window-1):int(window-1+n_train+14)]

  df_exog14_current = df_exog14[exo_features]
  col_train = df_sarimax14.loc[df_current_old14.index][0:n_train]
  exo_train = df_exog14_current.loc[df_current_old14.index][0:n_train]
  exo_test = df_exog14_current.loc[df_current_old14.index][n_train:]

  model = SARIMAX(col_train, exog=exo_train, order=(p,1,4), seasonal_order=(3,1,1,7), enforce_stationarity=False, enforce_invertibility=False)
  model_fit = model.fit(disp=False)
  forecast = model_fit.predict(len(col_train), len(col_train)+13, exog = exo_test)

  df_current = pd.DataFrame(forecast, columns=[str(exo_features)])

  return df_current

In [ ]:
def window_results(col_core, exo_features, n_train, n_test, p):
  temp_results = pd.DataFrame()
  for i in range(n_test):
    window = i + 1
    predict_current = sarimax_window(col_core, exo_features, n_train, n_test, p, window)
    predict_current = predict_current.rename(columns={str(exo_features):"window_"+str(window)})
    temp_results = pd.concat([temp_results, predict_current], axis=1)
  return temp_results

In [ ]:
def output_h(temp_results, h, n_test):
  n_iter = n_test-h+1
  df_results = []
  if h==1:
    for i in range(int(n_iter)):
      window = i + 1
      current_results = temp_results[['window_'+str(window)]].dropna()
      current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
      df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())
  if h>1:
    for i in range(int(h-1)):
      window = i + 1
      current_results = temp_results[['window_'+str(window)]].dropna()
      current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
      df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())
    for i in range(int(n_iter)):
      window = i + 1
      current_results = temp_results[['window_'+str(window)]].dropna()
      current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
      df_results.append(pd.DataFrame(current_results.iloc[int(h-1)]).transpose())
  df_results_python = pd.concat(df_results,axis=0)
  return df_results_python

In [ ]:
def predict_table(temp_results, n_test):
  df_final_forecast = pd.DataFrame()
  for h in range(1,15):
    df_current = output_h(temp_results, h, n_test)
    df_final_forecast = pd.concat([df_final_forecast, df_current], axis=1)
  return df_final_forecast

In [ ]:
# Function to calculate smape values
def smape(y_true, y_pred):
  m = len(y_true)
  n = 0
  sum = 0.0
  e = y_true - y_pred
  for i in range(0,m):
      denom = np.abs(y_true[i]) + np.abs(y_pred[i])
      if (denom != 0.0):
          sum = sum + np.abs(e[i]) / denom
          n = n + 1
  smape = 200 * sum / n
  return smape

In [ ]:
def smape_results(col_core, df_final_forecast, exo_features, p):
  df_US = pd.read_csv('COVID19_DATA.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  df_predict = np.exp(df_final_forecast)
  h_level = df_predict.columns.to_list()
  eval = []
  for cols in h_level:
    y_true = df_US.loc[df_predict.index][col_core]
    y_pred = df_predict[cols]
    value = smape(y_true, y_pred)
    eval.append(value)
  eval.append(np.mean(eval))
  df_results = pd.DataFrame(eval, columns=[str(exo_features)+"_p="+str(p)])
  h_level.append('Average')
  df_results.index = h_level
  return df_results

In [ ]:
for p in range(1,7):
  col_core = "new_deaths"
  n_train = 222
  n_test = 322
  all_features = ['icu_patients', 'hosp_patients', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated']
  for r in range(1,len(all_features)+1):
    new_feature_list = list(combinations(all_features,r))
    for items in new_feature_list:
      exo_features = list(items)
      temp_results = window_results(col_core, exo_features, n_train, n_test, p)
      df_final_forecast = predict_table(temp_results, n_test)
      df_results = smape_results(col_core, df_final_forecast, exo_features, p)

# **VAR**

In [ ]:
def var_window_results(col_core, h, window, n_train, n_test):
  mode = 'diff(log,7)'
  order = 1
  m = 7

  n_iter = int(n_test-h+1)

  # VAR select p
  def selectp(model):
    rows = []
    for i in [1,2,3,4]:
      result = model.fit(i)
      rows.append([i, result.bic])
    df_bic = pd.DataFrame(rows, columns=['lag','BIC'])
    df_lag = df_bic[df_bic.BIC == df_bic.BIC.min()]
    lag = int(df_lag['lag'])
    return lag

  # define df_US
  df_US = pd.read_csv('COVID19_DATA.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  df_US = df_US.drop(columns=['date'])
  df_US_log = pd.DataFrame()
  for col in df_US.columns:
    df_US_log[col] = df_US[col].apply(lambda x: log_transf(x))

  window_results = pd.DataFrame()
  features_list = ['icu_patients', 'hosp_patients']
  for r in range(1,3):
    window_results_current = pd.DataFrame()
    new_feature_list = list(combinations(features_list,r))
    for items in new_feature_list:
      all_list = list(items)
      all_list.append(col_core)

      df_data = pd.read_csv('/'+str(h)+'/'+str(window)+'_log.csv')
      df_data.index = pd.to_datetime(df_data['date'])
      df_data = df_data.drop(columns=['date'])
      set_index = df_data.index

      ### VAR
      df_cases_5state1 = df_US_log[all_list]
      df_cases_5state = df_cases_5state1.loc[set_index]
      df_cases_5state_train = df_cases_5state[0:n_train]
      df_cases_5state_test = df_cases_5state[n_train:]
      n_test_current = len(df_cases_5state_test)

      df_var_diff = df_cases_5state.diff(periods=m).dropna()

      df_var_train_diff, df_var_test_diff = df_var_diff[0:-n_test_current], df_var_diff[-n_test_current:]

      model = VAR(df_var_train_diff)
      lag = selectp(model)
      cases_model_fitted = model.fit(lag)
      forecast_input = df_var_train_diff.values[-lag:]
      fc, lwl, upl = cases_model_fitted.forecast_interval(y=forecast_input, steps=n_test_current, alpha=0.05)
      df_forecast = pd.DataFrame(fc, index=df_var_test_diff.index, columns=df_cases_5state.columns)

      pred_var = pd.DataFrame(df_forecast)[[col_core]]
      df_train = df_cases_5state_train[[col_core]]
      df_results_var = invert_transformation_period(df_train, pred_var, m)
      df_current = df_results_var.rename(columns={col_core:'var_'+ str(items)})
      window_results_current = pd.concat([window_results_current, df_current], axis=1)
    window_results = pd.concat([window_results, window_results_c

In [ ]:
def output_final_var(col_core, h, n_train, n_test):
  n_iter = n_test-h+1

  df_results = []

  if h==1:
    for i in range(int(n_iter)):
      window = i + 1
      df_results.append(var_window_results(col_core, h, window, n_train, n_test))

  if h>1:
    for i in range(int(h-1)):
      window = i + 1
      current_results = var_window_results(col_core, h, window, n_train, n_test)
      df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())
    for i in range(int(n_iter)):
      window = i + 1
      current_results = var_window_results(col_core, h, window, n_train, n_test)
      df_results.append(pd.DataFrame(current_results.iloc[int(h-1)]).transpose())

  df_results_python = pd.concat(df_results,axis=0)
  return df_results_python

In [ ]:
def predict_var(col_core, n_train, n_test):
  df_icu = pd.DataFrame()
  df_hosp = pd.DataFrame()
  df_icu_hosp = pd.DataFrame()
  df_columns = var_window_results(col_core='new_deaths', h=1, window=1, n_train=222, n_test=322).columns.to_list()
  for h in range(1,15):
    df_current_h = output_final_var(col_core, h, n_train, n_test)
    df_icu_current = df_current_h[[df_columns[0]]].rename(columns={df_columns[0]:"h="+str(h)})
    df_hosp_current = df_current_h[[df_columns[1]]].rename(columns={df_columns[1]:"h="+str(h)})
    df_icu_hosp_current = df_current_h[[df_columns[2]]].rename(columns={df_columns[2]:"h="+str(h)})
    df_icu = pd.concat([df_icu,df_icu_current], axis=1)
    df_hosp = pd.concat([df_hosp,df_hosp_current], axis=1)
    df_icu_hosp = pd.concat([df_icu_hosp,df_icu_hosp_current], axis=1)
  return df_icu, df_hosp, df_icu_hosp

In [ ]:
def smape_results(col_core, df_final_forecast, exo_varibale):
  df_US = pd.read_csv('COVID19_DATA.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  df_predict = np.exp(df_final_forecast)
  h_level = df_predict.columns.to_list()
  eval = []
  for cols in h_level:
    y_true = df_US.loc[df_predict.index][col_core]
    y_pred = df_predict[cols]
    value = smape(y_true, y_pred)
    eval.append(value)
  eval.append(np.mean(eval))
  df_results = pd.DataFrame(eval, columns=[exo_varibale])
  h_level.append('Average')
  df_results.index = h_level
  return df_results

# **MCP**

In [ ]:
state_list = ['WA','AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA',
              'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI',
              'MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND',
              'OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA',
              'WV','WI','WY']

In [ ]:
def new_data_frame(df_current):
  df_US_log = pd.DataFrame()
  for col in df_current.columns:
    df_US_log[col] = df_current[col].apply(lambda x: log_transf(x))
  df_state_cases = df_US_log.copy()

  df_lag = 14

  df_positive1 = pd.DataFrame(df_state_cases['deaths'][df_lag:])
  n = df_positive1.shape[0] + df_lag
  df_positive1['date'] = df_positive1.index
  df_positive1.index = np.arange(1,n-(df_lag-1))
  for col in df_state_cases.columns:
    newcol = df_state_cases[col][:-df_lag]
    newcol.index = np.arange(1,n-(df_lag-1))
    df_positive1[col + '_l' + str(df_lag)] = newcol
  df_positive1.index = df_positive1['date']
  df_positive1 = df_positive1.drop(columns=['date'])

  df_positive_new_all1 = df_positive1.copy()

  for i in range(df_lag-1):
    llag = i + 2
    df_positive2 = pd.DataFrame(df_state_cases['deaths'][(df_lag+(llag-1)):])
    df_positive2['date'] = df_positive2.index
    df_positive2.index = np.arange(1,(n-(df_lag+(llag-2))))
    for col in df_state_cases.columns:
      newcol = df_state_cases[col][:-(df_lag+(llag-1))]
      newcol.index = np.arange(1,(n-(df_lag+(llag-2))))
      df_positive2[col + '_l' + str(df_lag+(llag-1))] = newcol

    df_positive2.index = df_positive2['date']
    df_positive2 = df_positive2.drop(columns=['date'])
    df_positive2 = df_positive2.drop(columns=['deaths'])

    df_positive_new_all1 = pd.concat([df_positive_new_all1, df_positive2], axis=1)
  
  return df_positive_new_all1

In [ ]:
def create_and_save_files(states, h, n_train, n_test, df_current):
  df_core = pd.read_csv("state_deaths_mcp.csv")
  df_core.index = pd.to_datetime(df_core['Date'])
  df_core = df_core.drop(columns=['Date'])
  df_US = df_core[28:-14]
  df_new = new_data_frame(df_current)
  df_data_log_total = df_new.loc[df_US.index]
  n_iter = n_test-h+1
  for i in range(int(n_iter)):
    dataset = df_data_log_total[int(i):int(i + n_train + h)]
    current_train = dataset[0:int(n_train)].dropna()

    corrs = []
    for col in current_train.columns:
      corr, _ = pearsonr(current_train['deaths'], current_train[col])
      corrs.append(corr)
    df_positive_new_corr = pd.DataFrame(corrs).transpose()
    df_positive_new_corr.columns = current_train.columns
    df = df_positive_new_corr.copy()

    top_n = 8
    features = [df.T[col].nlargest(top_n).index.tolist() for n, col in enumerate(df.T)][0]
    new_set = dataset[features]
    new_set.to_csv("/"+states+"/"+str(h)+"/"+str(i+1)+".csv")

In [ ]:
n_train = 228
n_test = 140

In [ ]:
%%R
library(forecast)
library(MARSS)
library(MASS)
library(glmnet)
library(ncvreg)

invert_transformation <- function(df_train, df_forecast, period) {
  n = length(df_forecast)
  for (i in 1:n) {
    nn = length(df_train)
    current_value = df_forecast[i] + df_train[nn-(period-1)]
    df_train = append(df_train, current_value)
  }
  m = length(df_train)
  df_forecast = df_train[(m-n+1):m]
  return(df_forecast)
}

output_current_window <- function(h, window, states, n_train, n_test, col_core) {
  mode <- 'diff(log,7)'
  order <- 1
  m <- 7
  n_iter <- n_test-h+1

  date_list <- c()
  pred_mcp <- c()
  df_reg_all <- read.csv(paste('/state_level/', states,'/',h,'/',window,'.csv', sep=''))
  n = nrow(df_reg_all)
  df_train_all <- df_reg_all[1:n_train,]
  df_test_all <- df_reg_all[(n_train+1):n,]
  n_test_current <- nrow(df_test_all)

  #### Univariate analysis ####
  df_train_uni <- df_train_all[,2]
  test_date <- df_test_all$Date
  true_value <- df_test_all[,2]

  #### Multivariate models ####
  df_reg <- df_reg_all[complete.cases(df_reg_all), ]

  a = 7
  b = a + 1
  c = a + 2

  df_reg_train <- df_train_all
  df_reg_test <- df_test_all

  df_reg_diff <- as.data.frame(lapply(df_reg[,2:c], diff, lag=m, difference=order))
  df_reg_diff['Date'] <- df_reg[-c(seq(1,m)),]$Date

  df_reg_train_diff <- df_reg_diff[1:(nrow(df_reg_diff)-n_test_current),]
  df_reg_test_diff <- df_reg_diff[(nrow(df_reg_diff)-(n_test_current-1)):nrow(df_reg_diff),]

  ## Regularization
  X_train <- model.matrix(as.formula(paste(col_core, "~.", collapse = "+")),df_reg_train_diff[,1:b])[,-1]
  y_train <- df_reg_train_diff[,1]
  X_test <- as.matrix(df_reg_test_diff[,2:b])
  y_test <- df_reg_test_diff[,1]

  # MCP
  cv_output3 <- cv.ncvreg(X_train, y_train, gamma = 2, penalty = "MCP", nfolds = 10)
  best_lam3 <- cv_output3$lambda.min
  pred_mcp1 <- predict(cv_output3, X = X_test)

  date_list <- test_date
  pred_mcp <- invert_transformation(df_train_uni, pred_mcp1, m)

  df_forecast_R <- cbind.data.frame(date_list, exp(pred_mcp))

  return(df_forecast_R)
}

output_final_results = function(col_core, h, states, n_train, n_test) {
  n_iter <- n_test-h+1
  
  n_row = n_test
  n_col = 2
  
  df_results = data.frame(matrix(ncol = n_col, nrow = n_row))
  colnames(df_results) <- c('Date','MCP')
  
  if (h==1) {
    for (i in 1:n_iter) {
      df_results[i,] = output_current_window(h, i, states, n_train, n_test, col_core) 
    }
  }
  
  if (h==2) {
    i = 1
    df_results[i,] = output_current_window(h, i, states, n_train, n_test, col_core)[1,]
    df_results[i+(h-1),] = output_current_window(h, i, states, n_train, n_test, col_core)[h,]
    
    for (i in 1:n_iter) {
      df_results[i+(h-1),] = output_current_window(h, i, states, n_train, n_test, col_core)[h,]
    }
  }
  
  if (h >= 3) {
    for (i in 1:(h-1)){
      df_results[i,] = output_current_window(h, i, states, n_train, n_test, col_core)[1,]
    }
    
    for (i in 1:n_iter) {
      df_results[i+(h-1),] = output_current_window(h, i, states, n_train, n_test, col_core)[h,]
    }
  }
  return(df_results)
}

combine_results <- function(col_core, states, n_test, n_train){
  df_results = data.frame(matrix(ncol = 15, nrow = n_test))
  colnames(df_results) <- c('Date','h=1','h=2','h=3','h=4','h=5','h=6',
                            'h=7','h=8','h=9','h=10','h=11','h=12','h=13',
                            'h=14')
  h = 1
  outputdata = output_final_results(col_core, h, states, n_train, n_test)
  df_results[,1] = outputdata$Date
  df_results[,2] = outputdata$MCP
  
  for (h in 2:14) {
    outputdata = output_final_results(col_core, h, states, n_train, n_test)
    df_results[,h+1] = outputdata$MCP
  }
  
  return(df_results)
}